In [24]:
import pandas as pd
from datetime import datetime,date,timedelta
import numpy as np

In [ ]:
# Created dataframes from bitcoin currency data,the bitcoin transactions data and the suspicious bank transactions data, that was given in  the github page.
# Tided dataframes and created new columns for further usage.
# Created new dataframes that contains information about bank transactions and bitcoin transactions with similiar transaction amount
# made a day prior, a day after, in a day of a bitcoin transaction.
# Made a function that takes a column of interest and shows how many times appear column's vallues, that appear more than 1 time,
# and this vallues in dataframe.

In [34]:
# The data was collected from Yahoo Finance
btc_usd=pd.read_csv("C:/Users/omary/Downloads/BTC-USD (1).csv")
btc_usd.drop(["Open","High","Low","Close","Volume"],axis=1,inplace=True)
btc_usd.rename(columns={"Adj Close":"Price"},inplace=True)
btc_usd["Date"]=btc_usd.apply(lambda row: datetime.strptime(str(row["Date"]),"%Y-%m-%d"),axis=1)
print(btc_usd.head())

        Date       Price
0 2015-01-02  315.032013
1 2015-01-03  281.082001
2 2015-01-04  264.195007
3 2015-01-05  274.473999
4 2015-01-06  286.188995


In [29]:
# Made DataFrame with information from bitcoin transaction file.
# Row 12136 cause the Error, set up to ignore it. 
btc_df= pd.read_csv("btc-tx.csv", error_bad_lines=False,low_memory=False)

# Row 12136 cause the Error, set up to ignore it. 
# Converted time to datetime type for easier usage.
btc_df["time"]=btc_df.apply(lambda row: datetime.strptime(str(row["time"]),"%Y-%m-%d"),axis=1)

# Tided dataframe btc_df column "Transaction_amount_BTC" vallues and converted to float type.
btc_df.replace({"Transaction_amount_BTC":r'(\')+'}, {"Transaction_amount_BTC": r''},inplace=True, regex=True)
btc_df.replace({"Transaction_amount_BTC":r'(\")+'}, {"Transaction_amount_BTC": r''},inplace=True, regex=True)
btc_df.replace({"Transaction_amount_BTC":r'[a-z]+'}, {"Transaction_amount_BTC": 0},inplace=True, regex=True)
btc_df["Transaction_amount_BTC"]= pd.to_numeric(btc_df["Transaction_amount_BTC"], downcast="float").round()

# Created 2 new columns "BTC_USD" and "USD", where "BTC_USD" vallues is a price of Bitcoin in USD
# for correspond date
# And "USD" vallues transaction amount in USD that have been converted in match to "BTC_USD" column vallues.
llist=[]
for row in btc_df["time"]:
    x= btc_usd["Price"][btc_usd["Date"]==row].tolist()
    llist.append(float(x[-1]))
btc_df["BTC_USD"]=llist
btc_df["USD"]=btc_df.apply(lambda row: row["Transaction_amount_BTC"]* row["BTC_USD"],axis=1 )

print(btc_df.info())

b'Skipping line 12136: expected 5 fields, saw 13\n'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380398 entries, 0 to 380397
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   time                    380398 non-null  datetime64[ns]
 1   Hash                    380398 non-null  object        
 2   Sender                  380398 non-null  object        
 3   Receiver                380398 non-null  object        
 4   Transaction_amount_BTC  380398 non-null  float32       
 5   BTC_USD                 380398 non-null  float64       
 6   USD                     380398 non-null  float64       
dtypes: datetime64[ns](1), float32(1), float64(2), object(3)
memory usage: 18.9+ MB
None


In [40]:
#Create the dataframe from the bank transactions data.
transaction=pd.read_csv(r"C:\Users\omary\Downloads\download_data_fincen_files\download_transactions_map.csv")
transaction=transaction.dropna(subset=["begin_date","end_date"])
transaction["end_date"]=transaction.apply(lambda row: datetime.strptime(str(row["end_date"]),"%b %d, %Y"),axis=1)
transaction["begin_date"]=transaction.apply(lambda row: datetime.strptime(str(row["begin_date"]),"%b %d, %Y"),axis=1)

# Limited transaction rows due to differences in dates with btc_df dataframe.
transaction=transaction[transaction["end_date"]>=min(btc_df["time"])]

# Made 3 new columns that would merge on with btc_df. 
transaction["list_date"]=transaction.apply(lambda row: pd.period_range(row["end_date"]-timedelta(days=2),row["end_date"]+timedelta(days=2),freq="D").tolist(),axis=1)
transaction["day_before"]= transaction.apply(lambda row: row["list_date"][1].to_timestamp(),axis=1)
transaction["day_after"]= transaction.apply(lambda row: row["list_date"][3].to_timestamp(),axis=1)
print(transaction.head())

         id  icij_sar_id                 filer_org_name_id  \
0    223254         3297  the-bank-of-new-york-mellon-corp   
1    223255         3297  the-bank-of-new-york-mellon-corp   
232  223986         2765               societe-generale-sa   
233  223991         3062                       ofg-bancorp   
241  224028         3064  the-bank-of-new-york-mellon-corp   

                        filer_org_name begin_date   end_date  \
0    The Bank of New York Mellon Corp. 2015-03-25 2015-09-25   
1    The Bank of New York Mellon Corp. 2015-03-30 2015-09-25   
232                Société Générale SA 2016-07-28 2016-07-28   
233                        OFG Bancorp 2015-05-26 2015-05-26   
241  The Bank of New York Mellon Corp. 2015-04-16 2015-04-16   

                   originator_bank_id                    originator_bank  \
0                    cimb-bank-berhad                   CIMB Bank Berhad   
1                    cimb-bank-berhad                   CIMB Bank Berhad   
232  societe-g

In [50]:
# DataFrame that was created by merging of bitcoin transaction dataframe and bank transaction dataframe 
# on date in btc_df and 1 day prior corresponding date in transaction df.
df_1day_prior_bank_trns=pd.merge_ordered(btc_df,transaction,fill_method="ffill",left_on="time", right_on="day_before", how="outer")

# Made the dataframe where shows only rows of the merged dataframe, where an amount of bitcoin transaction and bank transaction
# set up to be different in 10% more or less.
result_1d_prior_bank_trns=df_1day_prior_bank_trns[(((df_1day_prior_bank_trns["USD"]/df_1day_prior_bank_trns["amount_transactions"]) >= 0.9) & ((df_1day_prior_bank_trns["USD"]/df_1day_prior_bank_trns["amount_transactions"]) <= 1.1))]

# Rearrenged columns for much readability.
result_1d_prior_bank_trns=result_1d_prior_bank_trns[['time', 'Hash', 'Sender', 'Receiver', 'Transaction_amount_BTC',
       'BTC_USD', 'USD', 'id','number_transactions', 'amount_transactions', 'icij_sar_id', 'filer_org_name_id',
       'filer_org_name', 'begin_date', 'end_date', 'originator_bank_id',
       'originator_bank', 'originator_bank_country',
       'beneficiary_bank_id', 'beneficiary_bank', 'beneficiary_bank_country' ]]
print(result_1d_prior_bank_trns)

             time                                               Hash  \
403    2015-01-27  62cddc726a155de23718927607ff2bec56f04743a372a2...   
514    2015-02-19  63f4a354ee3465b9d8702ae554801e34227076468d52f6...   
1179   2015-03-08  bb3ee99d573095dd4391d9adaa3d2e2390b242ed32d97b...   
1202   2015-03-08  1f448c659669856f2b0b1f869aa496ff85f16a0a3274b1...   
1230   2015-03-08  5f54fd3690a39b1182b6c50858bbb4f0d409cce6d96527...   
...           ...                                                ...   
476865 2017-09-01  c1cab12b4975f7a6957005cd8789c3921509cf35a85ba1...   
486478 2017-09-01  dce8cdff433123a5bcbaae5fcc064c5cdfa6bef377de7b...   
489659 2017-09-01  1468ec68a97fa232b386d13a2f3d62dd875d343b7b032e...   
498748 2017-09-01  916b8d1d0c53ec661ee07807a7b595cc7ef3a606352fc6...   
499063 2017-09-01  9240a7cc1ef4674cc0d50ddda2dbba660d885e09520aa2...   

                                                   Sender  \
403                    39ngTt6DWQRUbzjmx6ucxz7pPvQcm52CL3   
514     3Dbsz

In [49]:
# DataFrame that was created by merging of bitcoin transaction dataframe and bank transaction dataframe 
# on date in btc_df and 1 day after corresponding date in transaction df.
df_1day_after_bank_trns=pd.merge_ordered(btc_df,transaction,fill_method="ffill",left_on="time", right_on="day_after", how="outer")

# Made the dataframe where shows only rows of the merged dataframe, where an amount of bitcoin transaction and bank transaction
# set up to be different in 10% more or less.
result_1d_after_bank_trns=df_1day_after_bank_trns[(((df_1day_after_bank_trns["USD"]/df_1day_after_bank_trns["amount_transactions"]) >= 0.9) & ((df_1day_after_bank_trns["USD"]/df_1day_after_bank_trns["amount_transactions"]) <= 1.1))]

# Rearrenged columns for much readability.
result_1d_after_bank_trns=result_1d_after_bank_trns[['time', 'Hash', 'Sender', 'Receiver', 'Transaction_amount_BTC',
       'BTC_USD', 'USD', 'id','number_transactions', 'amount_transactions', 'icij_sar_id', 'filer_org_name_id',
       'filer_org_name', 'begin_date', 'end_date', 'originator_bank_id',
       'originator_bank', 'originator_bank_country',
       'beneficiary_bank_id', 'beneficiary_bank', 'beneficiary_bank_country' ]]

print(result_1d_after_bank_trns)

             time                                               Hash  \
306    2015-01-24  70429ff0821be5c74571ff1c9ad0a2ad1e44509a846399...   
320    2015-01-24  7d0524e60bb913b5b787ee62a80dfe141b88ad140b1f52...   
421    2015-01-27  8554239ab888eff343ed78b48204122383713bd6aab437...   
556    2015-02-23  97832849ceb9ec507a92aa1bc48304642cb75f848dad84...   
1333   2015-03-10  8206cfd1c005544c223946f921fc986b21fef60702a7b0...   
...           ...                                                ...   
293951 2017-09-01  c1cab12b4975f7a6957005cd8789c3921509cf35a85ba1...   
303564 2017-09-01  dce8cdff433123a5bcbaae5fcc064c5cdfa6bef377de7b...   
306745 2017-09-01  1468ec68a97fa232b386d13a2f3d62dd875d343b7b032e...   
315834 2017-09-01  916b8d1d0c53ec661ee07807a7b595cc7ef3a606352fc6...   
316149 2017-09-01  9240a7cc1ef4674cc0d50ddda2dbba660d885e09520aa2...   

                                                   Sender  \
306                    3KBUuGko4H5ke7EVsq9B7PLK1c5Askdd7y   
320          

In [47]:
# DataFrame that was created by merging of bitcoin transaction dataframe and bank transaction dataframe 
# on date in btc_df and in day of corresponding date in transaction df.
df_in_day_of_bank_trns=pd.merge_ordered(btc_df,transaction,fill_method="ffill",left_on="time", right_on="end_date", how="outer")

# Made the dataframe where shows only rows of the merged dataframe, where an amount of bitcoin transaction and bank transaction
# set up to be different in 10% more or less.
result_in_day_of_bank_trns=df_in_day_of_bank_trns[(((df_in_day_of_bank_trns["USD"]/df_in_day_of_bank_trns["amount_transactions"]) >= 0.9) & ((df_in_day_of_bank_trns["USD"]/df_in_day_of_bank_trns["amount_transactions"]) <= 1.1))]

# Rearrenged columns for much readability.
result_in_day_of_bank_trns= result_in_day_of_bank_trns[['time', 'Hash', 'Sender', 'Receiver', 'Transaction_amount_BTC',
       'BTC_USD', 'USD', 'id','number_transactions', 'amount_transactions', 'icij_sar_id', 'filer_org_name_id',
       'filer_org_name', 'begin_date', 'end_date', 'originator_bank_id',
       'originator_bank', 'originator_bank_country',
       'beneficiary_bank_id', 'beneficiary_bank', 'beneficiary_bank_country' ]]

print(result_in_day_of_bank_trns)


             time                                               Hash  \
160    2015-01-15  1a3ada97bedb4dba626cf53e2167e7f299e3826c67a3e7...   
162    2015-01-15  f65d59846e03a888ff9c5294733915e16c0d46a5062ac7...   
178    2015-01-15  465d1a1f2ecd394de21d2924d3388e10a604b398f3ad28...   
180    2015-01-15  93d2744621157833cb9435a3c8f015eb7679885915492d...   
186    2015-01-15  69596167f436c3b4e3a08cf22e87e3d61ef7716219b3b3...   
...           ...                                                ...   
532832 2017-09-01  c1cab12b4975f7a6957005cd8789c3921509cf35a85ba1...   
542445 2017-09-01  dce8cdff433123a5bcbaae5fcc064c5cdfa6bef377de7b...   
545626 2017-09-01  1468ec68a97fa232b386d13a2f3d62dd875d343b7b032e...   
554715 2017-09-01  916b8d1d0c53ec661ee07807a7b595cc7ef3a606352fc6...   
555030 2017-09-01  9240a7cc1ef4674cc0d50ddda2dbba660d885e09520aa2...   

                                                   Sender  \
160                    1CvBmeNZY7wgg5fyxfNd6vJzHN6EyhrHPs   
162          

In [52]:
# Made function that takes dataframe and colomn of interest, and give column's vallues that appear in the dataframe
# multiple times in a count vallues dataframe and in the orginal dataframe.
def show_multiple_appearing_parameter(df,column):
    val_count_df= df[column].value_counts().to_frame()
    if not val_count_df.empty:
        print(val_count_df[val_count_df!=1].dropna())
    for x in val_count_df[val_count_df!=1].dropna().index:
            print(df[df[column]==x])
    else:
        print("No parameters appear multiple times")
# As an example I used the function with dtaframe: result_in_day_of_bank_trns and column:"Receiver"   
show_multiple_appearing_parameter(result_in_day_of_bank_trns,"Receiver")


                                    Receiver
32MKGhKdTqEQZrrtd7iemEETz8bn7CJEaW       6.0
1Kr6QSydW9bFQG1mXiPNNu6WpJGmUa9i1g       6.0
3KBUuGko4H5ke7EVsq9B7PLK1c5Askdd7y       5.0
3FbZrzjRNYvFjRkaqXckd7gq2trufGpB4d       4.0
36HVwZeXkyQ4DzK4r4L5jXMyEoRwKXSyum       3.0
...                                      ...
1CabE2iNA6WDHPRN8BHwBbhQuzg8Qh65yD       2.0
14RFzpS5JwNv7wTeTLRqEEQpAds6PmFXCu       2.0
15orE7wnMxmrYEpUkC1HNJjaUYtUcRzECx       2.0
35AML3FjUygKkGRcQQaacEVj9eX5TWj1X6       2.0
1JAq1VBEC5xbuMbLdG4ujsgP14ivCgQRWB       2.0

[64 rows x 1 columns]
             time                                               Hash  \
107927 2016-05-06  ca204cf25c99ee5cc89532e420dbbbd14d3aacb6cd3fb0...   
108060 2016-05-11  1ea3c9bd4687c7427b5528555c2c4df79766576c542e09...   
262476 2016-10-17  e81cddbe2f5263aee830a89c1ec828d0670fc0de6bc4f7...   
271242 2016-11-22  d25a6247f1d232d5037d927702c2bf771f58c88950fe29...   
478986 2017-01-05  47a347544346d6466b495aa3cf123cf35197e5713584ef...   
47903

             time                                               Hash  \
478104 2016-12-29  784e7197ccbc782b99f84091fa9a73bf40f5bdd8d2da79...   
480429 2017-01-13  0ef152b2b2cb6d4501584c104368d7d1e12be87478e827...   
480543 2017-01-15  a1c60f74eb659a3235716b5e80737562806c940aa5572d...   

                                                   Sender  \
478104  3LftPV4FcEdMRan1GEfXXSfCuNXpBtRM7c "38y4ztLT8E...   
480429                 3LftPV4FcEdMRan1GEfXXSfCuNXpBtRM7c   
480543                 3LftPV4FcEdMRan1GEfXXSfCuNXpBtRM7c   

                                  Receiver  Transaction_amount_BTC  \
478104  36HVwZeXkyQ4DzK4r4L5jXMyEoRwKXSyum                  1600.0   
480429  36HVwZeXkyQ4DzK4r4L5jXMyEoRwKXSyum                  2000.0   
480543  36HVwZeXkyQ4DzK4r4L5jXMyEoRwKXSyum                  2000.0   

           BTC_USD           USD        id  number_transactions  \
478104  973.497009  1.557595e+06  235190.0                  2.0   
480429  823.984009  1.647968e+06  237423.0         

             time                                               Hash  \
105684 2016-04-01  825d31c7a9dfbfba0200fb1e844418941e6fb0dc7306c5...   
105710 2016-04-01  825d31c7a9dfbfba0200fb1e844418941e6fb0dc7306c5...   

                                                   Sender  \
105684  3P3LXtymthf9Z38CqTsBPu8FYDzE6xDhn4 3MyCd2bbU9A...   
105710  3P3LXtymthf9Z38CqTsBPu8FYDzE6xDhn4 "3MyCd2bbU9...   

                                  Receiver  Transaction_amount_BTC  \
105684  14WV5t42bBdrYKiYMuE53xVYDAiRknPihm                  5000.0   
105710  14WV5t42bBdrYKiYMuE53xVYDAiRknPihm                  5000.0   

           BTC_USD          USD        id  number_transactions  \
105684  417.959991  2089799.955  234744.0                  1.0   
105710  417.959991  2089799.955  234744.0                  1.0   

        amount_transactions  ...                 filer_org_name_id  \
105684            2103000.0  ...  the-bank-of-new-york-mellon-corp   
105710            2103000.0  ...  the-bank-of-new

             time                                               Hash  \
111403 2016-06-20  65259423d681b4575b4ea69db469add3e2dab6f945144a...   
111405 2016-06-20  65259423d681b4575b4ea69db469add3e2dab6f945144a...   

                                    Sender  \
111403  1536qwM6MgRakCLtTTte7YR3W1cpo4SKFP   
111405  1536qwM6MgRakCLtTTte7YR3W1cpo4SKFP   

                                  Receiver  Transaction_amount_BTC  \
111403  1NFUFFmT7XjFWAFx7GNsTc3tDTPvssudDj                  2086.0   
111405  1NFUFFmT7XjFWAFx7GNsTc3tDTPvssudDj                  2086.0   

           BTC_USD           USD        id  number_transactions  \
111403  737.226013  1.537853e+06  227832.0                  1.0   
111405  737.226013  1.537853e+06  241471.0                  1.0   

        amount_transactions  ...                 filer_org_name_id  \
111403            1586492.0  ...  the-bank-of-new-york-mellon-corp   
111405            1586492.0  ...  the-bank-of-new-york-mellon-corp   

                    

             time                                               Hash  \
478078 2016-12-29  1660c9ddc93566ed6e939a1b9a2f1d8cdc9b220b350c46...   
478080 2016-12-29  1660c9ddc93566ed6e939a1b9a2f1d8cdc9b220b350c46...   

                                    Sender  \
478078  19gATeKzQ2ya3UjAUWwx2MbNNCDXkKJ5T3   
478080  19gATeKzQ2ya3UjAUWwx2MbNNCDXkKJ5T3   

                                  Receiver  Transaction_amount_BTC  \
478078  13TQ7obyFMCj74sFsA19aQiFdptFZM6N4Z                  2550.0   
478080  13TQ7obyFMCj74sFsA19aQiFdptFZM6N4Z                  2550.0   

           BTC_USD           USD        id  number_transactions  \
478078  973.497009  2.482417e+06  235193.0                  1.0   
478080  973.497009  2.482417e+06  235196.0                  7.0   

        amount_transactions  ...                 filer_org_name_id  \
478078           2515695.00  ...  the-bank-of-new-york-mellon-corp   
478080           2661823.94  ...  the-bank-of-new-york-mellon-corp   

                    

             time                                               Hash  \
110023 2016-06-02  e8491ed4f08145b22b61ed4e2483856ff00f628d6faf40...   
111796 2016-06-22  599d6d4d3ebf9131d7a6eaa7548d78331abba9306ac2a7...   

                                                   Sender  \
110023  1AV9FWTQwcaDRqEFmiaC5xQJ7QKnARpj6R "12ucu9bHLe...   
111796  1Jj6Zr6hHyygYe6VPNVz2fSStfBcCARbZC "1MjLoap8Zt...   

                                  Receiver  Transaction_amount_BTC  \
110023  3J5KeQSVBUEs3v2vEEkZDBtPLWqLTuZPuD                  1928.0   
111796  3J5KeQSVBUEs3v2vEEkZDBtPLWqLTuZPuD                  4956.0   

           BTC_USD           USD        id  number_transactions  \
110023  537.971985  1.037210e+06  229403.0                  6.0   
111796  596.116028  2.954351e+06  227835.0                  9.0   

        amount_transactions  ...                 filer_org_name_id  \
110023            1101111.0  ...      china-investment-corporation   
111796            3128412.9  ...  the-bank-of-